<a href="https://colab.research.google.com/github/byoscar88888/SAA_entrega/blob/main/MIA/pdf1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install transformers torch
%pip install langchain
%pip install langchain_hub
%pip install langchain -q
%pip install langchain_community
%pip install accelerate
%pip install chromadb



In [ ]:
# Importar la clase HuggingFacePipeline de la biblioteca langchain
from langchain import HuggingFacePipeline
# Creación de una plantilla de prompt con LangChain
from langchain import PromptTemplate
# Importar la clase PDFMinerLoader y CharacterTextSplitter de LangChain
from langchain.document_loaders import PDFMinerLoader
from langchain.text_splitter import CharacterTextSplitter
# Importar y crear las incrustaciones usando HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
# Importar y crear la tienda de vectores Chroma
from langchain.vectorstores import Chroma
# Crear una cadena de recuperación conversacional utilizando LangChain
from langchain.chains import ConversationalRetrievalChain
# Utilizar herramientas de LangChain
from langchain.agents import load_tools, initialize_agent, AgentType


In [ ]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(model_id="stabilityai/stablelm-zephyr-3b",
                                        task="text-generation",
                                        model_kwargs={"temperature": 0.0, "max_length": 2048},
                                        pipeline_kwargs={"max_new_tokens": 2000})

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configura

In [ ]:
# Creación de una plantilla de prompt con LangChain
from langchain import PromptTemplate

# Definir una plantilla de prompt
template = """{prompt|>}{question}<|endoftext|><|assistant|>"""

prompt = PromptTemplate(template=template, input_variables=["question"])


In [ ]:
# Importar la clase PDFMinerLoader y CharacterTextSplitter de LangChain
from langchain.document_loaders import PDFMinerLoader
from langchain.text_splitter import CharacterTextSplitter

# Cargar y dividir el PDF en documentos
loader = PDFMinerLoader("C:/Users/User/Desktop/1911.01547v2.pdf")
pdf = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
documents = text_splitter.split_documents(pdf)

print(f"Number of documents: {len(documents)}")

Created a chunk of size 1995, which is longer than the specified 1024
Created a chunk of size 1435, which is longer than the specified 1024
Created a chunk of size 1167, which is longer than the specified 1024
Created a chunk of size 1344, which is longer than the specified 1024
Created a chunk of size 1037, which is longer than the specified 1024
Created a chunk of size 1484, which is longer than the specified 1024
Created a chunk of size 1057, which is longer than the specified 1024
Created a chunk of size 1045, which is longer than the specified 1024
Created a chunk of size 1120, which is longer than the specified 1024
Created a chunk of size 1268, which is longer than the specified 1024
Created a chunk of size 1464, which is longer than the specified 1024
Created a chunk of size 1898, which is longer than the specified 1024
Created a chunk of size 1066, which is longer than the specified 1024
Created a chunk of size 1506, which is longer than the specified 1024
Created a chunk of s

Number of documents: 211


In [ ]:

# Importar y crear las incrustaciones usando HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

# Crear el objeto de incrustaciones
embeddings = HuggingFaceEmbeddings()

# Incrustar la primera página del contenido de los documentos
query_result = embeddings.embed_query(documents[0].page_content)
print(query_result)


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[-0.042428407818078995, 0.021917276084423065, -0.022802453488111496, 0.008711552247405052, -0.022152915596961975, 0.002340551232919097, 0.06916218250989914, 0.006181136704981327, 0.02682526223361492, 0.03189503774046898, 0.024567363783717155, 0.03437989950180054, 0.014492090791463852, 0.004250028636306524, 0.05395049601793289, -0.03422139957547188, 0.011204037815332413, -0.021362343803048134, 0.03636736422777176, 0.002981957048177719, -0.08053705096244812, -0.03264548256993294, -0.017528433352708817, -0.001179322018288076, 0.026897743344306946, -0.03333449363708496, 0.07448337972164154, -0.05435512214899063, 0.02542339265346527, -0.03565740957856178, -0.01615114137530327, 0.030895398929715157, -0.06922626495361328, 0.01675441302359104, 2.145482312698732e-06, -0.04243778809905052, 0.012081839144229889, -0.0049616629257798195, -0.007090847473591566, 0.00020150988711975515, 0.025804486125707626, 0.07215629518032074, -0.006474302615970373, 0.02600269950926304, -0.04068383574485779, 0.02189

In [ ]:

# Importar y crear la tienda de vectores Chroma
from langchain.vectorstores import Chroma

# Crear la tienda de vectores utilizando los documentos e incrustaciones
vectorstore = Chroma.from_documents(documents, embeddings)

# Crear una cadena de recuperación conversacional utilizando LangChain
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(),
    return_source_documents=True
)

In [ ]:

# Ejemplo de uso de la cadena de recuperación conversacional
chat_history = []
query = "What is the definition of intelligence?"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

# Mostrar el contenido de la página del primer documento fuente
print(result['source_documents'][0].page_content)

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

I.2 Deﬁning intelligence: two divergent visions

Looked at in one way, everyone knows what
intelligence is; looked at in another way, no
one does.

Robert J. Sternberg, 2000

Many formal and informal deﬁnitions of intelligence have been proposed over the past
few decades, although there is no existing scientiﬁc consensus around any single deﬁnition.
Sternberg & Detterman noted in 1986 [87] that when two dozen prominent psychologists
were asked to deﬁne intelligence, they all gave somewhat divergent answers. In the context
of AI research, Legg and Hutter [53] summarized in 2007 no fewer than 70 deﬁnitions from
the literature into a single statement: “Intelligence measures an agent’s ability to achieve
goals in a wide range of environments.”

9
1
0
2

v
o
N
5
2

]
I

A
.
s
c
[

2
v
7
4
5
1
0
.
1
1
9
1
:
v
i
X
r
a

On the Measu

In [ ]:
%pip install --upgrade pip

In [ ]:
%pip install langchain pikepdf

In [ ]:
# Utilizar herramientas de LangChain
from langchain.agents import load_tools, initialize_agent, AgentType

# Cargar herramientas adicionales
tools = load_tools(["arxiv"])

# Inicializar el agente
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
agent_chain.run(
    "What's the paper On the measure of intelligence, by François Chollet, about?",
)

#Funciona el problema es que no sabemos el porque no se termina de ejecutar del todo, es decir, funcoina al minuto de estar ejecutado y nos da la propia información pero despues es como un bucle que nunca acaba, el error que aparece es simplemente porque lo he parado a mano.

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

arxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [arxiv]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What's the paper On the measure of intelligence, by François Choll

KeyboardInterrupt: 